### How to load yaml cam configs and access info

- supply one config function to access cam dicts for specified property
- access functions in cam modules to access cam properties (noise, preampgain,etc)

#### cam dict loading

- keep it simple: load yamls in sequence from most general to most user specific directory
- while walking through those directories construct a master dict that just gets updated with new info and would thus overwrite existing entries with the most user specific ones taking precendence

Something like:

```
def config.loadCamInfo():
    camdict = {}
    for dir in config_cam_dirs:
        for cam_yaml in find_yamls(dir): # perhaps we allow several files that get merged but not necessary
            camdict.update(yaml.safe_load(cam_yaml)
    return camdict
```

#### access functions

- camera backend specific access functions that consult both hardcoded info and config cam properties to retrieve required info (precedence is config file over hardcoded)
- examples below in fakecam classes


### How to install site wide config files

Here I suggest adopting a simple approach as we do for the plugin packages. A site-config repo would contain the cam config files (yaml) and an `install_cams.py` script that copies the relevant camera file(s) to the desired level config directory. (very much similar to the `install_plugins.py` script)

### mock functionality that would go in some fashion into PYME.config

In [1]:
import yaml
import glob

# here we read the two files we made (for Andor emCCD and sCMOS) from the info in the hardcoded dicts
# in reality we would need to walk through the config directories

# this could be a cached read that gets triggered the first time getCamProps is used
camProps = {}
for yamlfile in glob.glob('*.yaml'):
    with open(yamlfile,'r') as fi:
        camProps.update(yaml.safe_load(fi))

def config_getCamProps(serno,prop):
    # possibly generate some warnings for certain cases, e.g. serno exists but property does not
    if serno in camProps:
        try:
            return camProps[serno]['properties'][prop]
        except KeyError:
            return None
    else:
        return None

### Heterogeneous dictionary structure that is adopted

- each camera has an entry (dict-like) accessed by its serial number
- the interesting stuff is in the `properties` slot
- the `properties` slot has, based an camera type, different sub entries
- by convention `name` and `model` slots are present to help identify cam type for the reader
- other slots can be added, e.g. a `comment` slot if so desired, for now they are completely ignored

Example of sCMOS type entry:

In [2]:
print(yaml.dump( camProps['VSC-00954'], default_flow_style=False, default_style='' ))

model: Zyla
name: Andor sCMOS
properties:
  noiseProperties:
    12-bit (high well capacity):
      ADOffset: 100
      ElectronsPerCount: 6.97
      ReadNoise: 5.96
      SaturationThreshold: 2047
    12-bit (low noise):
      ADOffset: 100
      ElectronsPerCount: 0.28
      ReadNoise: 1.1
      SaturationThreshold: 2047
    16-bit (low noise & high well capacity):
      ADOffset: 100
      ElectronsPerCount: 0.5
      ReadNoise: 1.33
      SaturationThreshold: 65535



Example of emCCD type entry:

In [3]:
print(yaml.dump( camProps[1823], default_flow_style=False, default_style='' ))

model: unknown
name: Andor IXON
properties:
  noiseProperties:
    ADOffset: 971
    DefaultEMGain: 150
    ElectronsPerCount: 27.32
    NGainStages: 536
    ReadNoise: 109.8
    SaturationThreshold: 16383
  preampGain: 0



### A couple of fake cams to exercise the cam property lookup

The `@property` bits suggest possible implementations of the lookup mechanism

In [4]:
class fakecamCMOS(object):
    _noise_properties = {} # no hardcoded cams
    
    def __init__(self):
        self.serno = 'CSC-00425'
        pass
    
    def GetSerialNumber(self):
        return self.serno
    
    def SetSerialNumber(self, serno):
        self.serno = serno
    
    def GetSimpleGainMode(self):
        return '12-bit (low noise)'

    @property
    def noise_properties(self):
        """return the noise properties for a the given camera

        """
        serno = self.GetSerialNumber()
        np = config_getCamProps(serno,'noiseProperties')
        if np is not None:
            try:
                return np[self.GetSimpleGainMode()]
            except KeyError:
                logger.warn('unable to load noiseproperties for gain mode "%s" from camera config entry' % self.GetSimpleGainMode())
        # if we get to here fall back to hardcoded cameras
        try:
            return self._noise_properties[serno][self.GetSimpleGainMode()]
        except KeyError: # last resort is default set of values
            logger.warn('camera specific noise props not found - using default noise props')
            return {'ReadNoise' : 1.1,
                    'ElectronsPerCount' : 0.28,
                    'ADOffset' : 100,
                    'SaturationThreshold' : 2**11-1
                    }


In [5]:
fc = fakecamCMOS()

In [6]:
config_getCamProps(fc.GetSerialNumber(),'noiseProperties')

{'12-bit (low noise)': {'ADOffset': 100,
  'ElectronsPerCount': 0.45,
  'ReadNoise': 1.21,
  'SaturationThreshold': 1776},
 '16-bit (high dynamic range)': {'ADOffset': 100,
  'ElectronsPerCount': 1.08,
  'ReadNoise': 1.84,
  'SaturationThreshold': 44185}}

In [7]:
fc.noise_properties

{'ADOffset': 100,
 'ElectronsPerCount': 0.45,
 'ReadNoise': 1.21,
 'SaturationThreshold': 1776}

In [8]:
class fakecamEmCCD(object):
    _noise_properties = {
        4532 : { #Gain setting of 1
        'ReadNoise' : 56.1,
        'ElectronsPerCount' : 3.6,
        'NGainStages' : 345,
        'ADOffset' : 203,
        'DefaultEMGain' : 90,
        'SaturationThreshold' : 5.4e4#(2**16 -1)
        }
    } # no hardcoded cams
    _preamp_gains = {
        4532 : 1
    }
    
    def __init__(self):
        self.serno = 1823
        pass
    
    def GetSerialNumber(self):
        return self.serno
    
    def SetSerialNumber(self, serno):
        self.serno = serno
    
    @property
    def noise_properties(self):
        """return the noise properties for a the given camera

        """
        serno = self.GetSerialNumber()
        np = config_getCamProps(serno,'noiseProperties')
        if np is not None:
            return np

        # if we get to here fall back to hardcoded cameras
        try:
            return self._noise_properties[serno]
        except KeyError: # we give up if we can't find the noise props
            raise RuntimeError('camera specific noise props not found for serial no %d' % serno)

    @property
    def preampGain(self):
        serno = self.GetSerialNumber()
        pag = config_getCamProps(serno,'preampGain')
        if pag is not None:
            return pag
        
        # if we get to here fall back to hardcoded cameras
        try:
            return self._preamp_gains[serno]
        except KeyError: # last resort is default set of values
            return 2 # default value
        

In [9]:
fce = fakecamEmCCD()
print(fce.preampGain)
fce.noise_properties

0


{'ADOffset': 971,
 'DefaultEMGain': 150,
 'ElectronsPerCount': 27.32,
 'NGainStages': 536,
 'ReadNoise': 109.8,
 'SaturationThreshold': 16383}

In [10]:
fce.SetSerialNumber(5414)
print(fce.preampGain)
fce.noise_properties

0


{'ADOffset': 413,
 'DefaultEMGain': 90,
 'ElectronsPerCount': 25.24,
 'NGainStages': 536,
 'ReadNoise': 61.33,
 'SaturationThreshold': 16383}

In [11]:
# now access the hardcoded cam
fce.SetSerialNumber(4532)
print(fce.preampGain)
fce.noise_properties

1


{'ADOffset': 203,
 'DefaultEMGain': 90,
 'ElectronsPerCount': 3.6,
 'NGainStages': 345,
 'ReadNoise': 56.1,
 'SaturationThreshold': 54000.0}